In [1]:
%reload_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# Change main system path to be able to run code from src folder
import sys
p = sys.path[0]
# Mac OS
if sys.path[0].endswith('/notebooks'):
    main_path = p[:-len('/notebooks')]
if sys.path[0].endswith('/techdoc/content'):
    main_path = p[:-len('/techdoc/content')]
    
# Windows OS
if sys.path[0].endswith('\\notebooks'): 
    main_path = p[:-len('\\notebooks')]
if sys.path[0].endswith('\\techdoc\content'): 
    main_path = p[:-len('\\techdoc\content')]

sys.path[0] = main_path

In [6]:
import itertools
from pathlib import Path
from pprint import pprint

from matplotlib import pyplot as plt, cm
import numpy as np
import pandas as pd
from sklearn.preprocessing import minmax_scale
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from tqdm import tqdm

from src import config

pd.set_option("max_colwidth", 80)
RANDOM_SEED = 42  # For reproducibility

In [7]:
metadata = pd.read_csv(config.DATA_DIR + "metadata.csv", index_col="sample_id")
metadata.head()

,split,instrument_type,features_path,features_md5_hash
sample_id,,,,
S0000,train,commercial,train_features/S0000.csv,017b9a71a702e81a828e6242aa15f049
S0001,train,commercial,train_features/S0001.csv,0d09840214054d254bd49436c6a6f315
S0002,train,commercial,train_features/S0002.csv,3f58b3c9b001bfed6ed4e4f757083e09
S0003,train,commercial,train_features/S0003.csv,e9a12f96114a2fda60b36f4c0f513fb1
S0004,train,commercial,train_features/S0004.csv,b67603d3931897bfa796ac42cc16de78


In [8]:
train_files = metadata[metadata["split"] == "train"]["features_path"].to_dict()
val_files = metadata[metadata["split"] == "val"]["features_path"].to_dict()
test_files = metadata[metadata["split"] == "test"]["features_path"].to_dict()

print("Number of training samples: ", len(train_files))
print("Number of validation samples: ", len(val_files))
print("Number of testing samples: ", len(test_files))

Number of training samples:  766
Number of validation samples:  293
Number of testing samples:  511


In [10]:
train_labels = pd.read_csv(config.DATA_DIR + "train_labels.csv", index_col="sample_id")
train_labels.head()

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
sample_id,,,,,,,,,,
S0000,0,0,0,0,0,0,0,0,1,0
S0001,0,1,0,0,0,0,0,0,0,0
S0002,0,0,0,0,0,1,0,0,0,0
S0003,0,1,0,1,0,0,0,0,1,0
S0004,0,0,0,1,0,1,1,0,0,0


# PREPROCESSING

In [11]:
def drop_frac_and_He(df):
    """
    Drops fractional m/z values, m/z values > 100, and carrier gas m/z

    Args:
        df: a dataframe representing a single sample, containing m/z values

    Returns:
        The dataframe without fractional an carrier gas m/z
    """

    # drop fractional m/z values
    df = df[df["m/z"].transform(round) == df["m/z"]]
    assert df["m/z"].apply(float.is_integer).all(), "not all m/z are integers"

    # drop m/z values greater than 99
    df = df[df["m/z"] < 100]

    # drop carrier gas
    df = df[df["m/z"] != 4]

    return df

In [12]:
def remove_background_abundance(df):
    """
    Subtracts minimum abundance value

    Args:
        df: dataframe with 'm/z' and 'abundance' columns

    Returns:
        dataframe with minimum abundance subtracted for all observations
    """

    df["abundance_minsub"] = df.groupby(["m/z"])["abundance"].transform(
        lambda x: (x - x.min())
    )

    return df

In [13]:
def scale_abun(df):
    """
    Scale abundance from 0-100 according to the min and max values across entire sample

    Args:
        df: dataframe containing abundances and m/z

    Returns:
        dataframe with additional column of scaled abundances
    """

    df["abun_minsub_scaled"] = minmax_scale(df["abundance_minsub"].astype(float))

    return df

In [14]:
# Preprocess function
def preprocess_sample(df):
    df = drop_frac_and_He(df)
    df = remove_background_abundance(df)
    df = scale_abun(df)
    return df

# FEATURE ENGINEERING

In [15]:
# Create a series of temperature bins
temprange = pd.interval_range(start=-100, end=1500, freq=100)
temprange

# Make dataframe with rows that are combinations of all temperature bins
# and all m/z values
allcombs = list(itertools.product(temprange, [*range(0, 100)]))

allcombs_df = pd.DataFrame(allcombs, columns=["temp_bin", "m/z"])
allcombs_df.head()

,temp_bin,m/z
0,"(-100, 0]",0
1,"(-100, 0]",1
2,"(-100, 0]",2
3,"(-100, 0]",3
4,"(-100, 0]",4


In [16]:
def abun_per_tempbin(df):

    """
    Transforms dataset to take the preprocessed max abundance for each
    temperature range for each m/z value

    Args:
        df: dataframe to transform

    Returns:
        transformed dataframe
    """

    # Bin temperatures
    df["temp_bin"] = pd.cut(df["temp"], bins=temprange)

    # Combine with a list of all temp bin-m/z value combinations
    df = pd.merge(allcombs_df, df, on=["temp_bin", "m/z"], how="left")

    # Aggregate to temperature bin level to find max
    df = df.groupby(["temp_bin", "m/z"]).max("abun_minsub_scaled").reset_index()

    # Fill in 0 for abundance values without information
    df = df.replace(np.nan, 0)

    # Reshape so each row is a single sample
    df = df.pivot_table(columns=["m/z", "temp_bin"], values=["abun_minsub_scaled"])

    return df

In [17]:
# Assembling preprocessed and transformed training set

train_features_dict = {}
print("Total number of train files: ", len(train_files))

for i, (sample_id, filepath) in enumerate(tqdm(train_files.items())):

    # Load training sample
    temp = pd.read_csv(config.DATA_DIR + filepath)

    # Preprocessing training sample
    train_sample_pp = preprocess_sample(temp)

    # Feature engineering
    train_sample_fe = abun_per_tempbin(train_sample_pp).reset_index(drop=True)
    train_features_dict[sample_id] = train_sample_fe

train_features = pd.concat(
    train_features_dict, names=["sample_id", "dummy_index"]
).reset_index(level="dummy_index", drop=True)

Total number of train files:  766


100%|██████████| 766/766 [02:42<00:00,  4.71it/s]


In [18]:
train_features.head()

m/z              0                                                         \
temp_bin  (-100, 0]  (0, 100] (100, 200] (200, 300] (300, 400] (400, 500]   
sample_id                                                                   
S0000           0.0  0.004085   0.004641   0.001394   0.000188   0.000000   
S0001           0.0  0.000000   0.000000   0.000000   0.000000   0.000000   
S0002           0.0  0.002270   0.002495   0.001688   0.000636   0.000597   
S0003           0.0  0.001336   0.002464   0.001099   0.000992   0.000676   
S0004           0.0  0.005993   0.012429   0.009380   0.005099   0.006921   

m/z                                                    ...         99  \
temp_bin  (500, 600] (600, 700] (700, 800] (800, 900]  ... (500, 600]   
sample_id                                              ...              
S0000       0.000000   0.000000   0.000000   0.000000  ...   0.000000   
S0001       0.000000   0.000000   0.000000   0.000000  ...   0.000000   
S0002       0.000819   0.000155   0.000235   0.000227  ...   0.000001   
S0003       0.000883   0.000892   0.000631   0.000361  ...   0.000005   
S0004       0.001966   0.000499   0.000880   0.000767  ...   0.000005   

m/z                                                                     \
temp_bin     (600, 700] (700, 800] (800, 900] (900, 1000] (1000, 1100]   
sample_id                                                                
S0000      0.000000e+00   0.000000   0.000000    0.000000     0.000000   
S0001      0.000000e+00   0.000000   0.000000    0.000000     0.000000   
S0002      9.397170e-07   0.000001   0.000001    0.000001     0.000000   
S0003      4.693477e-06   0.000004   0.000004    0.000004     0.000003   
S0004      3.174421e-06   0.000004   0.000004    0.000003     0.000004   

m/z                                                            
temp_bin  (1100, 1200] (1200, 1300] (1300, 1400] (1400, 1500]  
sample_id                                                      
S0000              0.0          0.0          0.0          0.0  
S0001              0.0          0.0          0.0          0.0  
S0002              0.0          0.0          0.0          0.0  
S0003              0.0          0.0          0.0          0.0  
S0004              0.0          0.0          0.0          0.0  

[5 rows x 1600 columns]

In [19]:
# Make sure that all sample IDs in features and labels are identical
assert train_features.index.equals(train_labels.index)

# PERFORM MODELING

In [20]:
# Define stratified k-fold validation
skf = StratifiedKFold(n_splits=10, random_state=RANDOM_SEED, shuffle=True)

# Define log loss
log_loss_scorer = make_scorer(log_loss, needs_proba=True)

In [21]:
# Check log loss score for baseline dummy model
def logloss_cross_val(clf, X, y):

    # Generate a score for each label class
    log_loss_cv = {}
    for col in y.columns:

        y_col = y[col]  # take one label at a time
        log_loss_cv[col] = np.mean(
            cross_val_score(clf, X.values, y_col, cv=skf, scoring=log_loss_scorer)
        )

    avg_log_loss = np.mean(list(log_loss_cv.values()))

    return log_loss_cv, avg_log_loss

In [22]:
# Define logistic regression model
logreg_clf = LogisticRegression(
    penalty="l1", solver="liblinear", C=10, random_state=RANDOM_SEED
)
print("Logistic regression model log-loss:\n")
logreg_logloss = logloss_cross_val(logreg_clf, train_features, train_labels)
pprint(logreg_logloss[0])
print("Average log-loss")
logreg_logloss[1]

Logistic regression model log-loss:

{'basalt': 0.27667494614302185,
 'carbonate': 0.261405355676578,
 'chloride': 0.2842770277471818,
 'iron_oxide': 0.38245389629312065,
 'oxalate': 0.004103320117163466,
 'oxychlorine': 0.2990978185052554,
 'phyllosilicate': 0.40751189988765824,
 'silicate': 0.35176540325465755,
 'sulfate': 0.34463431214084367,
 'sulfide': 0.147159858336199}
Average log-loss


0.2759083838101679

## Training the model on all of the data

In [23]:
# Train logistic regression model with l1 regularization, where C = 10

# Initialize dict to hold fitted models
fitted_logreg_dict = {}

# Split into binary classifier for each class
for col in train_labels.columns:

    y_train_col = train_labels[col]  # Train on one class at a time

    # output the trained model, bind this to a var, then use as input
    # to prediction function
    clf = LogisticRegression(
        penalty="l1", solver="liblinear", C=10, random_state=RANDOM_SEED
    )
    fitted_logreg_dict[col] = clf.fit(train_features.values, y_train_col)  # Train

# SUBMISSION

In [24]:
# Create dict with both validation and test sample IDs and paths
all_test_files = val_files.copy()
all_test_files.update(test_files)
print("Total test files: ", len(all_test_files))

Total test files:  804


In [25]:
# Import submission format
submission_template_df = pd.read_csv(config.DATA_DIR + "submission_format.csv", index_col="sample_id"
)
compounds_order = submission_template_df.columns
sample_order = submission_template_df.index
sample_order

In [48]:
def predict_for_sample(sample_id, fitted_model_dict):

    # Import sample
    temp_sample = pd.read_csv(config.DATA_DIR + all_test_files[sample_id])

    # Preprocess sample
    temp_sample = preprocess_sample(temp_sample)

    # Feature engineering on sample
    temp_sample = abun_per_tempbin(temp_sample)

    # Generate predictions for each class
    temp_sample_preds_dict = {}

    for compound in compounds_order:
        clf = fitted_model_dict[compound]
        temp_sample_preds_dict[compound] = clf.predict_proba(temp_sample.values)[:, 1][0]

    return temp_sample_preds_dict

In [49]:
ht, ht_preds= predict_for_sample('S0766', fitted_logreg_dict)
ht

m/z                       0                                             \
temp_bin           (-100, 0] (0, 100] (100, 200] (200, 300] (300, 400]   
abun_minsub_scaled       0.0   0.4101   0.593334   0.606578   0.678006   

m/z                                                                        \
temp_bin           (400, 500] (500, 600] (600, 700] (700, 800] (800, 900]   
abun_minsub_scaled   0.553591   0.596561   0.606902   0.535487    0.69642   

m/z                 ...         99                                   \
temp_bin            ... (500, 600] (600, 700] (700, 800] (800, 900]   
abun_minsub_scaled  ...    0.00138   0.001495   0.001755   0.001762   

m/z                                                                    \
temp_bin           (900, 1000] (1000, 1100] (1100, 1200] (1200, 1300]   
abun_minsub_scaled    0.001607     0.001233          0.0          0.0   

m/z                                           
temp_bin           (1300, 1400] (1400, 1500]  
abun_minsub_scaled          0.0          0.0  

[1 rows x 1600 columns]

In [50]:
ht_preds

{'basalt': 1.8749238063968804e-07,
 'carbonate': 6.360979933728063e-05,
 'chloride': 3.561453567104628e-06,
 'iron_oxide': 1.1502322156104167e-07,
 'oxalate': 5.209063517706726e-05,
 'oxychlorine': 8.91375456756441e-05,
 'phyllosilicate': 8.675169387164811e-08,
 'silicate': 0.9987708495559746,
 'sulfate': 1.945849050435298e-06,
 'sulfide': 2.288561830579e-06}

In [33]:
# Dataframe to store submissions in
final_submission_df = pd.DataFrame(
    [
        predict_for_sample(sample_id, fitted_logreg_dict)
        for sample_id in tqdm(sample_order)
    ],
    index=sample_order,
)



  0%|          | 0/804 [00:24<?, ?it/s]































































































































































































































































































































































































































































































































































































































































































































































































100%|██████████| 804/804 [02:58<00:00,  4.51it/s]


In [34]:
final_submission_df

,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
sample_id,,,,,,,,,,
S0766,1.874924e-07,6.360980e-05,3.561454e-06,1.150232e-07,0.000052,0.000089,8.675169e-08,9.987708e-01,1.945849e-06,0.000002
S0767,2.259988e-01,2.619353e-01,7.582974e-03,3.366967e-01,0.000131,0.226646,1.466800e-02,8.796766e-01,5.817046e-02,0.009801
S0768,5.743828e-01,5.084393e-01,8.049057e-04,2.186054e-01,0.000768,0.277291,8.996323e-01,6.861487e-01,2.505327e-01,0.002382
S0769,1.907595e-02,9.231807e-02,4.238754e-02,7.070841e-02,0.000083,0.977444,1.084093e-01,8.636899e-02,3.698815e-01,0.003077
S0770,8.435578e-04,8.440979e-03,4.847759e-02,3.103282e-01,0.000355,0.862540,5.698068e-01,3.194947e-03,1.443584e-03,0.019162
...,...,...,...,...,...,...,...,...,...,...
S1565,1.180406e-06,3.153381e-07,5.261932e-07,2.678724e-02,0.070783,0.081526,8.101057e-03,2.554537e-09,1.045407e-08,0.040918
S1566,4.458478e-02,3.730912e-02,5.762006e-02,3.045002e-02,0.000069,0.012209,6.927492e-05,3.116179e-02,5.047030e-02,0.000905
S1567,3.427490e-05,3.485583e-03,3.514940e-01,4.334896e-02,0.000388,0.000143,4.298960e-02,5.928919e-04,9.999916e-01,0.007115
